## Ausgabe
In GeoPandas verfügt man - je nach zusätzlich verwendeten Bibliotheken - über verschiedene Ausgabemöglichkeiten:  
- intern `plot()`
- [Matplotlib](https://matplotlib.org/): Universelle Pythonbibliothek zur Ausgabe von Diagrammen und Karten
- [Geoplot](https://residentmario.github.io/geoplot/index.html): hoch aggregierte Plotting Biliothek für räumliche Daten. Basiert auf [CartoPy](https://pypi.org/project/Cartopy/) and matplotlib
- [Folium](https://python-visualization.github.io/folium/): Ausgabebibliothek basierend auf leaflet.js

In [ ]:
import geopandas
#interner Datensatz GeoPandas: Bezirke New Yorks
df = geopandas.read_file(geopandas.datasets.get_path('nybb'))
ax = df.plot(figsize=(5, 5), alpha=0.5, edgecolor='k')

Mit [contextily](https://github.com/geopandas/contextily) können Tilemaps aus dem Internet geladen werden, z.B. OSM Tiles. Dazu müssen wir das Geodataframe erst umprojizieren in Web-Mercator (EPSG 3857).

In [ ]:
df = df.to_crs(epsg=3857)

In [ ]:
import contextily as ctx

In [ ]:
ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
ctx.add_basemap(ax)

### GeoPlot

In [ ]:
import geopandas
import geoplot

world = geopandas.read_file(
    geopandas.datasets.get_path('naturalearth_lowres')
)
boroughs = geopandas.read_file(
    geoplot.datasets.get_path('nyc_boroughs')
)
collisions = geopandas.read_file(
    geoplot.datasets.get_path('nyc_injurious_collisions')
)

Basisausgabe mit Geoplot:

In [ ]:
geoplot.polyplot(world, figsize=(8, 4))

Geoplot kann Geodaten in jedes Koordinatensystem, das von CartoPy unterstützt wird umprojizieren, eine Auflistung findet sich [hier](https://scitools.org.uk/cartopy/docs/latest/crs/projections.html):

In [ ]:
# use the Orthographic map projection (e.g. a world globe)
ax = geoplot.polyplot(
    world, projection=geoplot.crs.Orthographic(), figsize=(8, 4)
)
ax.spines['geo'].set_visible(True)

`geoplot.polyplot` kann nur die übergebenen Geometrien darstellen. Sollen Farbwerte z.B. als Choropleth dargestellt werden, kann man das mit der [Mapclassify](https://pysal.org/mapclassify/) Bibliothek und `geoplot.choropleth` machen:

In [ ]:
import mapclassify
gpd_per_person = world['gdp_md_est'] / world['pop_est']
scheme = mapclassify.Quantiles(gpd_per_person, k=5)

# Note: this code sample requires geoplot>=0.4.0.
geoplot.choropleth(
    world, hue=gpd_per_person, scheme=scheme,
    cmap='Greens', figsize=(8, 4)
)

### Folium
Folium baut auf einem Python-Wrapper für leaflet.js auf. Es macht es einfach, Daten in Python mit minimalen Anweisungen zu visualisieren.  

In [ ]:
# Bibliotheken importieren
import pandas as pd
import geopandas
import folium
import matplotlib.pyplot as plt
from shapely.geometry import Point

#Daten aus CSV in ein Pandas Dataframe laden:
df1 = pd.read_csv('./data/volcano_data_2010.csv')
df = df1.loc[:, ("Year", "Name", "Country", "Latitude", "Longitude", "Type")]
df.info()

In [ ]:
#Nun erstellen wir eine Geometrie und konvertieren das 
geometry = geopandas.points_from_xy(df.Longitude, df.Latitude)
geo_df = geopandas.GeoDataFrame(df[['Year','Name','Country', 'Latitude', 'Longitude', 'Type']], geometry=geometry)

geo_df.head()

In [ ]:
#Hintergrundlayer und Unique Values finden
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
df.Type.unique()

Mit Matplotlib sieht das ganze dann so aus:

In [ ]:
fig, ax = plt.subplots(figsize=(24,18))
world.plot(ax=ax, alpha=0.4, color='grey')
geo_df.plot(column='Type', ax=ax, legend=True)
plt.title('Volcanoes')

Eine Karte mit Folium erstellen: es stehen verschiedene Hintergrundkarten zur Verfügung, z.B. `Stamen Terrain`

In [ ]:
map = folium.Map(location = [48,7.8], tiles = "Stamen Terrain", zoom_start = 9)
map

In [ ]:
# OpenStreetMap
map = folium.Map(location = [52.518611, 13.408333], tiles='OpenStreetMap' , zoom_start = 13)
map

Nun wollen wir die Karte der Vulkane mit unterschiedlichen Markern darstellen:

In [ ]:
#Für die Vulkankarte verwenden wir die Hintergrundkarte "Stamen Terrain"
map = folium.Map(location = [4,10], tiles = "Stamen Terrain", zoom_start = 3)

In [ ]:
# Um mehrere Marker einzufügen interieren wir über die Geometry des Geodataframes und erzeugen eine Liste mit Punkten
# add a different color marker associated with type of volcano

geo_df_list = [[point.y, point.x] for point in geo_df.geometry ]

i = 0
for coordinates in geo_df_list:
    #assign a color marker for the type of volcano, Strato being the most common
    if geo_df.Type[i] == "Stratovolcano":
        type_color = "green"
    elif geo_df.Type[i] == "Complex volcano":
        type_color = "blue"
    elif geo_df.Type[i] == "Shield volcano":
        type_color = "orange"
    elif geo_df.Type[i] == "Lava dome":
        type_color = "pink"
    else:
        type_color = "purple"


    #now place the markers with the popup labels and data
    map.add_child(folium.Marker(location = coordinates,
                            popup =
                            "Year: " + str(geo_df.Year[i]) + '<br>' +
                            "Name: " + str(geo_df.Name[i]) + '<br>' +
                            "Country: " + str(geo_df.Country[i]) + '<br>'
                            "Type: " + str(geo_df.Type[i]) + '<br>'
                            "Coordinates: " + str(geo_df_list[i]),
                            icon = folium.Icon(color = "{}".format(type_color))))
    i = i + 1

In [ ]:
map